In [3]:
import os
from tqdm import tqdm
import json
from exiftool import ExifToolHelper
from datetime import datetime
import math
import re

In [4]:
if not os.path.exists("output"):
    os.mkdir("output")
if not os.path.exists("takeout_folders"):
    os.mkdir("takeout_folders")

In [5]:
failures = []

In [4]:
# https://stackoverflow.com/a/52371976/3675086
def deg_to_dms(deg, type='lat'):
    decimals, number = math.modf(deg)
    d = int(number)
    m = int(decimals * 60)
    s = (deg - d - m / 60) * 3600.00
    compass = {
        'lat': ('N','S'),
        'lon': ('E','W')
    }
    compass_str = compass[type][0 if d >= 0 else 1]
    return {
        'dms': (abs(d), abs(m), abs(s)),
        'ref': compass_str
    }

In [13]:
def attach_metadata(files):
    # Iterate thru each provided file
    for file in files:
        # Attempt to find metadata for this file
        try:
            with open(f'{file}.json') as json_file:
                metadata = json.load(json_file)
        except FileNotFoundError:
            # If no metadata found, just raw copy to the output folder
            failures.append(file)
            import shutil
            shutil.copyfile(f"{file}", f'../../../../../failures/{file}')
            shutil.copyfile(f"{file}", f'../../../../../output/{file}')
        else:
            with open(file, 'rb') as img_file:
                img = Image(img_file)

            # Set timestmap metadata
            timestamp = datetime.utcfromtimestamp(int(metadata['photoTakenTime']['timestamp'])).strftime(DATETIME_STR_FORMAT)
            img.datetime = timestamp

            # Try to set original date, and if it fails cuz no original date was ever set on the photo, then just skip 
            try:
                img.datetime_original = timestamp
            except Exception as e:
                pass

            # Set make and model metadata if available
            if 'mobileUpload' in metadata['googlePhotosOrigin'] and 'deviceType' in metadata['googlePhotosOrigin']['mobileUpload']:
                if metadata['googlePhotosOrigin']['mobileUpload']['deviceType'] == "IOS_PHONE":
                    img.make = "Apple"
                    img.model = "iPhone 6"


            # Only add lat and lon to metadata if they are available from original photo 
            if metadata['geoData']['latitude'] != 0.0 and metadata['geoData']['longitude'] != 0.0:
                # convert decimal to degree, minute, second
                lat = deg_to_dms(metadata['geoData']['latitude'])
                lon = deg_to_dms(metadata['geoData']['longitude'])

                img.gps_latitude = lat['dms']
                img.gps_latitude_ref = lat['ref']
                img.gps_longitude = lon['dms']
                img.gps_longitude_ref = lon['ref']

            with open(f'../../../../../output/{file}', 'wb+') as new_image_file:
                new_image_file.write(img.get_file())

In [8]:
os.chdir("takeout_folders")
for folder in os.listdir():
    print(f"Fixing folder {folder}")

    backwards_traverse = "../"
    
    # Traverse down extra paths if necessary to get to year folders 
    if os.path.exists(f'{folder}/Takeout'):
        os.chdir(f"{folder}/Takeout/Google Photos")
        backwards_traverse = "../../../"

    # Fetch all exported photo folders
    photo_folders = [folders[1] for folders in os.walk("./")][0]
    
    for photo_folder in tqdm(photo_folders):
        os.chdir(photo_folder) # Go to folder

        # Fetch all image types in this folder, ensure not to match .json 
        image_files = [file for file in os.listdir() if re.search(r'.*\.(jpg|jpeg)$', file.lower(), re.IGNORECASE)]

        # TODO SUPPORT LODAING FOR PNGS 

        attach_metadata(image_files) # Attach metadata for all image files 

        # attach_metadata(glob.glob("*.MOV"))

        os.chdir("../") # Go up a folder to prepare for the next folder
    os.chdir(backwards_traverse) # Go up to the project main directory

Fixing folder takeout-20230525T041716Z-001


100%|██████████| 5/5 [00:03<00:00,  1.30it/s]


Fixing folder takeout-20230525T041716Z-002


100%|██████████| 2/2 [00:00<00:00,  2.15it/s]


In [9]:
len(failures)

357

In [12]:
video_file = "takeout_folders/takeout-20230525T041716Z-001/Takeout/Google Photos/Photos from 2017/IMG_0472.mp4"

attach_metadata([video_file])

received request


UnpackError: 

+--------+------------+-------+-------+------------------------+
| Offset | Access     | Value | Bytes | Format                 |
+--------+------------+-------+-------+------------------------+
|        |            |       |       | TiffHeader (Structure) |
| 0      | byte_order | 1281  | 05 01 | tiff_byte_order        |
+--------+------------+-------+-------+------------------------+

ValueError occurred during unpack operation:

1281 is not a valid TiffByteOrder